# Search Planet Basemap API
This script searches the basemap API for basemap grids across the circumpolar region.

## API and Package Set-Up

In [1]:
import os
import json
import requests
import urllib.request
import re
import numpy as np
import pandas as pd
import geopandas as gpd
import shapely as shp
from pprint import pprint

In [2]:
# Get Planet API Key
%load_ext dotenv
%dotenv

api_key = os.getenv('PL_BM_API_KEY')

## User Input

In [6]:
basemap_year = "2024"

## Search for Basemap

In [3]:
#setup Planet base URL
API_URL = "https://api.planet.com/basemaps/v1/mosaics"

#setup session
session = requests.Session()

#authenticate
session.auth = (api_key, "")

In [4]:
# make get request to access mosaic from basemaps API
res = session.get(API_URL)

#response status code - should be 200 if there were no errors
print(res.status_code)

#print metadata for mosaic
mosaic = res.json()
# print(json.dumps(mosaic, indent=2))


200


In [5]:
#make get request to access mosaic from basemaps API
res = session.get("https://api.planet.com/basemaps/v1/series")

#response status code
print(res.status_code)

#print names of mosaic options
series = res.json()
names = [item['name'] for item in series['series']]
pprint(names)

# print names of global Q3 (July-Sept) mosaics
mosaic_link = [
    mosaic['_links']['mosaics'] for mosaic in series['series'] if mosaic['name'] == 'Global Quarterly'
][0]
pprint(mosaic_link)
mosaics = session.get(mosaic_link).json()
mosaic_names = [mosaic['name'] for mosaic in mosaics['mosaics'] if re.match('^.*q3.*$', mosaic['name'])]
pprint(mosaic_names)
years = [re.search('\d{4}', mosaic)[0] for mosaic in mosaic_names]
years

<>:20: SyntaxWarning: invalid escape sequence '\d'
<>:20: SyntaxWarning: invalid escape sequence '\d'
/tmp/ipykernel_475188/1789921444.py:20: SyntaxWarning: invalid escape sequence '\d'
  years = [re.search('\d{4}', mosaic)[0] for mosaic in mosaic_names]


200
['Global Monthly',
 'Global Quarterly',
 'PS analytic_sr biweekly subscription',
 'PS analytic_sr daily subscription',
 'PS analytic_sr monthly subscription',
 'PS analytic_sr quarterly subscription',
 'PS analytic_sr twice_monthly subscription',
 'PS analytic_sr weekly subscription',
 'PS normalized_analytic biweekly subscription',
 'PS normalized_analytic daily subscription',
 'PS normalized_analytic monthly L16 subscription',
 'PS normalized_analytic monthly subscription',
 'PS normalized_analytic quarterly subscription',
 'PS normalized_analytic twice_monthly subscription',
 'PS normalized_analytic weekly subscription',
 'PS sen2_normalized_analytic quarterly subscription',
 'PS visual biweekly subscription',
 'PS visual daily subscription',
 'PS visual twice_monthly subscription',
 'PS visual weekly subscription']
'https://api.planet.com/basemaps/v1/series/060c47b7-57fb-429e-9e5b-bee1c89c6973/mosaics?api_key=PLAK0a0262a75eb34455bb250697655805f5'
['global_quarterly_2016q3_mosai

['2016', '2017', '2018', '2019', '2020', '2021', '2022', '2023', '2024']

In [8]:
mosaic_names = [item for item in mosaic_names if re.match('^.*' + basemap_year + '.*$', item)]
mosaic_names

['global_quarterly_2024q3_mosaic']

In [9]:
# set bounding box
string_bbox = '-180,40,180,85'

#search for mosaic quad using AOI
search_parameters = {
    'bbox': string_bbox,
    'minimal': True
}

In [10]:
item_name = []
item_id = []
link = []
percent_covered = []
geom = []
for mosaic_name in mosaic_names:

    #set params for search using name of mosaic
    parameters = {
        "name__is": mosaic_name
    }

    #make get request to access mosaic from basemaps API
    res = session.get(API_URL, params = parameters)

    #response status code
    print(res.status_code)

    #print metadata for mosaic
    mosaic = res.json()
    print(json.dumps(mosaic, indent=2))

    #get id
    mosaic_id = mosaic['mosaics'][0]['id']

    #get bbox for entire mosaic
    mosaic_bbox = mosaic['mosaics'][0]['bbox']

    print(mosaic_id)
    print(mosaic_bbox)

    #accessing quads using metadata from mosaic
    quads_url = "{}/{}/quads".format(API_URL, mosaic_id)

    #send request
    res = session.get(quads_url, params=search_parameters, stream=True)

    quads = res.json()
    pprint(quads)

    item_name = item_name + [mosaic_name]*len(quads['items'])
    item_id = item_id + [item['id'] for item in quads['items']]
    link = link + [item['_links']['download'] for item in quads['items']]
    percent_covered = percent_covered + [item['percent_covered'] for item in quads['items']]
    geom = geom + [
                shp.geometry.box(item['bbox'][0], item['bbox'][1], item['bbox'][2], item['bbox'][3]) 
                for item 
                in quads['items']
            ]
    # print(item_id)

    try:
        next_link = quads['_links']['_next']
        # print(next_link)
    except:
        next_link = None

    while next_link:
        res = session.get(next_link, stream=True)
        quads = res.json()

        item_name = item_name + [mosaic_name]*len(quads['items'])
        item_id = item_id + [item['id'] for item in quads['items']]
        link = link + [item['_links']['download'] for item in quads['items']]
        percent_covered = percent_covered + [item['percent_covered'] for item in quads['items']]
        geom = geom + [
            shp.geometry.box(item['bbox'][0], item['bbox'][1], item['bbox'][2], item['bbox'][3]) 
            for item 
            in quads['items']
        ]

        try:
            next_link = quads['_links']['_next']
            # print(next_link)
        except:
            next_link = None

200
{
  "_links": {
    "_self": "https://api.planet.com/basemaps/v1/mosaics?api_key=PLAK0a0262a75eb34455bb250697655805f5"
  },
  "mosaics": [
    {
      "_links": {
        "_self": "https://api.planet.com/basemaps/v1/mosaics/442e78c3-e3df-4133-94e4-0b77900192db?api_key=PLAK0a0262a75eb34455bb250697655805f5",
        "quads": "https://api.planet.com/basemaps/v1/mosaics/442e78c3-e3df-4133-94e4-0b77900192db/quads?api_key=PLAK0a0262a75eb34455bb250697655805f5&bbox={lx},{ly},{ux},{uy}",
        "tiles": "https://tiles.planet.com/basemaps/v1/planet-tiles/global_quarterly_2024q3_mosaic/gmap/{z}/{x}/{y}.png?api_key=PLAK0a0262a75eb34455bb250697655805f5"
      },
      "bbox": [
        -179.999999974944,
        -59.623325217855,
        179.999999975056,
        76.016093660188
      ],
      "coordinate_system": "EPSG:3857",
      "datatype": "byte",
      "first_acquired": "2024-07-01T00:00:00.000Z",
      "grid": {
        "quad_size": 4096,
        "resolution": 4.777314267823516
      },

In [11]:
circumpolar_grids = gpd.GeoDataFrame(
    data = pd.DataFrame({
        'name': item_name,
        'id': item_id,
        'link': link,
        'percent_covered': percent_covered,
    }),
    geometry = gpd.GeoSeries(
        data = geom,
        crs = 'EPSG:4326'
    )
)

circumpolar_grids

,name,id,link,percent_covered,geometry
0,global_quarterly_2024q3_mosaic,321-1707,https://link.planet.com/basemaps/v1/mosaics/44...,None,"POLYGON ((-123.39844 75.97355, -123.39844 76.0..."
1,global_quarterly_2024q3_mosaic,322-1707,https://link.planet.com/basemaps/v1/mosaics/44...,None,"POLYGON ((-123.22266 75.97355, -123.22266 76.0..."
2,global_quarterly_2024q3_mosaic,323-1707,https://link.planet.com/basemaps/v1/mosaics/44...,None,"POLYGON ((-123.04688 75.97355, -123.04688 76.0..."
3,global_quarterly_2024q3_mosaic,324-1707,https://link.planet.com/basemaps/v1/mosaics/44...,None,"POLYGON ((-122.87109 75.97355, -122.87109 76.0..."
4,global_quarterly_2024q3_mosaic,325-1707,https://link.planet.com/basemaps/v1/mosaics/44...,None,"POLYGON ((-122.69531 75.97355, -122.69531 76.0..."
...,...,...,...,...,...
526273,global_quarterly_2024q3_mosaic,1828-1272,https://link.planet.com/basemaps/v1/mosaics/44...,None,"POLYGON ((141.50391 39.90974, 141.50391 40.044..."
526274,global_quarterly_2024q3_mosaic,1829-1272,https://link.planet.com/basemaps/v1/mosaics/44...,None,"POLYGON ((141.67969 39.90974, 141.67969 40.044..."
526275,global_quarterly_2024q3_mosaic,1830-1272,https://link.planet.com/basemaps/v1/mosaics/44...,None,"POLYGON ((141.85547 39.90974, 141.85547 40.044..."
526276,global_quarterly_2024q3_mosaic,1831-1272,https://link.planet.com/basemaps/v1/mosaics/44...,None,"POLYGON ((142.03125 39.90974, 142.03125 40.044..."


In [13]:
circumpolar_grids.to_file('../data/circumpolar_basemap_grids_' + basemap_year + '.geojson')

'../data/arctic_boreal_basemap_grids2024.geojson'